In [1]:

# Create a folder for the raw images
!mkdir -p /kaggle/working/data/raw

# Unzip the .tgz file (Adjust the path if yours is slightly different)
!tar -xzf "/kaggle/input/flower-dataset-102/102flowers.tgz" -C /kaggle/working/data/raw

import os
import os
image_dir = "/kaggle/working/data/raw/jpg" # Check if tar created a 'jpg' subfolder
if not os.path.exists(image_dir):
    image_dir = "/kaggle/working/data/raw"

num_images = len([f for f in os.listdir(image_dir) if f.endswith('.jpg')])
print(f"Total images extracted: {num_images}")

Total images extracted: 8189


In [2]:
import os
import shutil
import scipy.io
from sklearn.model_selection import train_test_split


# --- KAGGLE PATH ADJUSTMENTS ---
# Replace 'oxford-102-flower-dataset' with the actual folder name from os.listdir('/kaggle/input')
DATASET_NAME = 'flower-dataset-102' 
RAW_DIR = "/kaggle/working/data/raw/jpg"  # Kaggle usually puts images in a 'jpg' folder
LABELS_FILE = "/kaggle/input/flower-dataset-102/imagelabels.mat"
OUTPUT_DIR = "/kaggle/working/data/splits"
# -------------------------------

def get_images_and_labels():
    """Load all images and labels"""
    # Get image paths - using full paths
    images = sorted(
        [os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.endswith(".jpg")]
    )

    # Load labels (convert from 1-102 to 0-101)
    labels = scipy.io.loadmat(LABELS_FILE)["labels"][0] - 1
    return images, labels

def create_split(images, labels, seed):
    """Create one train/val/test split"""
    train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(
        images, labels, test_size=0.5, random_state=seed, stratify=labels
    )

    val_imgs, test_imgs, val_labels, test_labels = train_test_split(
        temp_imgs, temp_labels, test_size=0.5, random_state=seed, stratify=temp_labels
    )

    split_dir = os.path.join(OUTPUT_DIR, f"split_{seed}")

    for split_name, imgs, lbls in [
        ("train", train_imgs, train_labels),
        ("val", val_imgs, val_labels),
        ("test", test_imgs, test_labels),
    ]:
        for img, label in zip(imgs, lbls):
            class_folder = os.path.join(split_dir, split_name, f"class_{label:03d}")
            os.makedirs(class_folder, exist_ok=True)
            # Use copy instead of copy2 if you run into permission issues on Kaggle
            shutil.copy(img, class_folder)

    print(f"Split {seed}: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")


images, labels = get_images_and_labels()
print(f"Total images: {len(images)}, Classes: {len(set(labels))}")

# Create 2 splits
create_split(images, labels, seed=42)
create_split(images, labels, seed=123)
print(f"\nDone! Splits saved in {OUTPUT_DIR}")



Total images: 8189, Classes: 102
Split 42: Train=4094, Val=2047, Test=2048
Split 123: Train=4094, Val=2047, Test=2048

Done! Splits saved in /kaggle/working/data/splits


Noiw we goign to preproccess   lets define how preproccess works first 

In [11]:
from PIL import Image
import numpy as np
from torchvision import transforms


def preprocess_for_vgg19():
    """
    Resize to 224x224
    Normalize pixel values
    Apply VGG19-specific preprocessing
    """
    # For TRAINING data (with augmentation)

    train_transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    # For VALIDATION/TEST data (NO augmentation)

    test_transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    return train_transform, test_transform


def preprocess_for_yolov5():
    """
    Resize to 640x640
    Normalize
    Apply YOLOv5-specific preprocessing
    """
    # For TRAINING data (with augmentation)

    train_transform = transforms.Compose(
        [
            transforms.Resize((244, 244)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    # For VALIDATION/TEST data (NO augmentation)

    test_transform = transforms.Compose(
        [
            transforms.Resize((244, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    return train_transform, test_transform


In [10]:
# Evaluation fucntion 

import matplotlib.pyplot as plt

def plot_cross_entropy_loss(history, model_name, split_name, test_loss):
    """
    Calculate accuracy
    Generate confusion matrix
    Plot accuracy and loss curves
    """

    # the losses and epochs lists
    train_losses = history["train_loss"]
    val_losses = history["val_loss"]

    epochs = range(1, len(history["train_loss"]) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, 'b-', label='Train Loss', linewidth=2)
    plt.plot(epochs, val_losses, 'g-', label='Validation Loss', linewidth=2)
    plt.axhline(y=test_loss, color='r', linestyle='--', label=f'Test Loss: {test_loss:.4f}', linewidth=2)
    plt.title(f'Cross-Entropy Loss - {model_name.upper()} ({split_name})', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    os.makedirs('results/graphs', exist_ok=True)
    save_path = f'results/graphs/{model_name}_{split_name}_loss.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()  # Close to free memory
    print(f"Saved: {save_path}")


def plot_accuracy_graph(history, model_name, split_name, test_acc):
    epochs = range(1, len(history["train_loss"]) + 1)

    train_acc = history["train_acc"]
    val_acc = history["val_acc"]

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_acc, 'b-', label='Train Accuracy', linewidth=2)
    plt.plot(epochs, val_acc, 'g-', label='Validation Accuracy', linewidth=2)
    plt.axhline(y=test_acc, color='r', linestyle='--', label=f'Test Accuracy: {test_acc:.2f}%', linewidth=2)
    plt.title(f'Accuracy - {model_name.upper()} ({split_name})', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy (%)', fontsize=12)
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    os.makedirs('results/graphs', exist_ok=True)
    save_path = f'results/graphs/{model_name}_{split_name}_accuracy.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()  # Close to free memory
    print(f"Saved: {save_path}")

print("defined plotting methods")

defined plotting methods


Imstall yolo

In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.7 MB/s eta 0:00:00a 0:00:01


**Load all whats needed **

In [6]:
import torch
import torch.nn as nn
from torchvision import models
from ultralytics import YOLO
import sys

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [27]:
# VGG19 - IMPROVED VERSION
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 102
EPOCHS = 35
LEARNING_RATE = 0.01


class VGG19Classifier(nn.Module):
    def __init__(self, num_classes=102):
        super().__init__()

        # Load pretrained VGG19
        self.backbone = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)

        # Freeze features (conv layers) only
        for param in self.backbone.features.parameters():
            param.requires_grad = False

        # Replace the last layer
        self.backbone.classifier[6] = nn.Linear(4096, num_classes)

        self.model = self.backbone.to(DEVICE)

        self.criterion = nn.CrossEntropyLoss()
        
        # Use SGD with momentum (often better for transfer learning)
        self.optimizer = torch.optim.SGD(
            self.model.classifier.parameters(),
            lr=LEARNING_RATE,
            momentum=0.9,
            weight_decay=1e-4
        )
        
        # Learning rate scheduler
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='max',
            factor=0.1,
            patience=3,
            verbose=True
        )

    def forward(self, x):
        return self.model(x)

    def train_model(self, train_loader=None, val_loader=None):
        self.train_loader = train_loader
        self.val_loader = val_loader

        self.history = {
            "train_loss": [],
            "train_acc": [],
            "val_loss": [],
            "val_acc": [],
        }

        best_val_acc = 0

        for epoch in range(EPOCHS):
            # Train
            self.model.train()
            train_loss = 0
            train_correct = 0

            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)

                self.optimizer.zero_grad()
                outputs = self.forward(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == labels).sum().item()

            train_acc = 100 * train_correct / len(self.train_loader.dataset)
            avg_train_loss = train_loss / len(self.train_loader)

            # Validate
            self.model.eval()
            val_correct = 0
            val_loss = 0
            with torch.no_grad():
                for images, labels in self.val_loader:
                    images, labels = images.to(DEVICE), labels.to(DEVICE)
                    outputs = self.forward(images)
                    val_correct += (outputs.argmax(1) == labels).sum().item()
                    val_loss += self.criterion(outputs, labels).item()

            val_acc = 100 * val_correct / len(self.val_loader.dataset)
            avg_val_loss = val_loss / len(self.val_loader)

            # Step scheduler based on validation accuracy
            self.scheduler.step(val_acc)

            # Track best
            if val_acc > best_val_acc:
                best_val_acc = val_acc

            self.history["train_loss"].append(avg_train_loss)
            self.history["train_acc"].append(train_acc)
            self.history["val_loss"].append(avg_val_loss)
            self.history["val_acc"].append(val_acc)

            # Get current learning rate
            current_lr = self.optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch+1}/{EPOCHS} - Train: {train_acc:.2f}% - Val: {val_acc:.2f}% - LR: {current_lr:.6f}")

        print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
        return self.history

    def test_model(self, test_loader=None):
        self.test_loader = test_loader
        self.model.eval()
        test_correct = 0
        test_loss = 0

        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.forward(images)
                test_correct += (outputs.argmax(1) == labels).sum().item()
                test_loss += self.criterion(outputs, labels).item()

        test_acc = 100 * test_correct / len(self.test_loader.dataset)
        avg_test_loss = test_loss / len(self.test_loader)

        print(f"\nTest Accuracy: {test_acc:.2f}%")
        return test_acc, avg_test_loss

print("success")

success


In [25]:
# VGG19 - WORKING VERSION
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 102
EPOCHS = 35

LEARNING_RATE = 0.001


class VGG19Classifier(nn.Module):
    def __init__(self, num_classes=102):
        super().__init__()

        # Load pretrained VGG19
        self.backbone = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)

        # Freeze features (conv layers) only
        for param in self.backbone.features.parameters():
            param.requires_grad = False

        # Replace ONLY the last layer (4096 → 102)
        self.backbone.classifier[6] = nn.Linear(4096, num_classes)

        self.model = self.backbone.to(DEVICE)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(
            self.model.classifier.parameters(),  # Train entire classifier
            lr=LEARNING_RATE
        )

        

    def forward(self, x):
        return self.model(x)

    def train_model(self, train_loader=None, val_loader=None):
        self.train_loader = train_loader
        self.val_loader = val_loader

        self.history = {
            "train_loss": [],
            "train_acc": [],
            "val_loss": [],
            "val_acc": [],
        }

        for epoch in range(EPOCHS):
            # Train
            self.model.train()
            train_loss = 0
            train_correct = 0

            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)

                self.optimizer.zero_grad()
                outputs = self.forward(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == labels).sum().item()

            train_acc = 100 * train_correct / len(self.train_loader.dataset)
            avg_train_loss = train_loss / len(self.train_loader)

            # Validate
            self.model.eval()
            val_correct = 0
            val_loss = 0
            with torch.no_grad():
                for images, labels in self.val_loader:
                    images, labels = images.to(DEVICE), labels.to(DEVICE)
                    outputs = self.forward(images)
                    val_correct += (outputs.argmax(1) == labels).sum().item()
                    val_loss += self.criterion(outputs, labels).item()

            val_acc = 100 * val_correct / len(self.val_loader.dataset)
            avg_val_loss = val_loss / len(self.val_loader)

            self.history["train_loss"].append(avg_train_loss)
            self.history["train_acc"].append(train_acc)
            self.history["val_loss"].append(avg_val_loss)
            self.history["val_acc"].append(val_acc)

            print(f"Epoch {epoch+1}/{EPOCHS} - Train: {train_acc:.2f}% - Val: {val_acc:.2f}%")

        return self.history

    def test_model(self, test_loader=None):
        self.test_loader = test_loader
        self.model.eval()
        test_correct = 0
        test_loss = 0

        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.forward(images)
                test_correct += (outputs.argmax(1) == labels).sum().item()
                test_loss += self.criterion(outputs, labels).item()

        test_acc = 100 * test_correct / len(self.test_loader.dataset)
        avg_test_loss = test_loss / len(self.test_loader)

        print(f"\nTest Accuracy: {test_acc:.2f}%")
        return test_acc, avg_test_loss

print("success")

success


In [15]:
# Add yolov5 to path (if you cloned it)
sys.path.append("../yolov5")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 102
EPOCHS = 20
LEARNING_RATE = 0.001
MODEL_WEIGHTS = "yolov5s-cls.pt"

class YOLOv5Classifier(nn.Module):
    def __init__(self, num_classes=102):
        super().__init__()

        # Load YOLOv5 CLASSIFICATION model from Torch Hub
        # Path should just be the filename; it will auto-download to /kaggle/working
        self.backbone = torch.hub.load(
            "ultralytics/yolov5",
            "custom",
            path=MODEL_WEIGHTS, 
            trust_repo=True,
        )

        # Freeze backbone (layers 0-8), keep head (layer 9) trainable
        freeze_layers = [
            "model.0.",
            "model.1.",
            "model.2.",
            "model.3.",
            "model.4.",
            "model.5.",
            "model.6.",
            "model.7.",
            "model.8.",
        ]

        for name, param in self.backbone.model.named_parameters():
            if any(layer in name for layer in freeze_layers):
                param.requires_grad = False  # Freeze backbone
            else:
                param.requires_grad = True  # Train head (layer 9)

        # Replace final linear layer: 1000 classes → 102 classes
        # model.9.linear is the classification layer
        in_features = self.backbone.model.model[9].linear.in_features  # 1280
        self.backbone.model.model[9].linear = nn.Linear(in_features, num_classes)

        self.model = self.backbone.to(DEVICE)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, self.model.parameters()), lr=LEARNING_RATE
        )

    def forward(self, x):
        return self.model(x)

    def train_model(self, train_loader=None, val_loader=None):

        # 0. Store data loaders
        self.train_loader = train_loader
        self.val_loader = val_loader

        # Store history for plotting later
        self.history = {
            "train_loss": [],
            "train_acc": [],
            "val_loss": [],
            "val_acc": [],
        }
        # ============================================
        # TRAINING LOOP
        # ============================================
        for epoch in range(EPOCHS):
            # Train
            self.model.train()
            train_loss = 0
            train_correct = 0

            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)

                self.optimizer.zero_grad()
                outputs = self.forward(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == labels).sum().item()

            train_acc = 100 * train_correct / len(self.train_loader.dataset)
            avg_train_loss = train_loss / len(self.train_loader)

            # Validate
            self.model.eval()
            val_correct = 0
            val_loss = 0
            with torch.no_grad():
                for images, labels in self.val_loader:
                    images, labels = images.to(DEVICE), labels.to(DEVICE)
                    outputs = self.forward(images)
                    val_correct += (outputs.argmax(1) == labels).sum().item()
                    # add loss calculation for validation
                    val_loss += self.criterion(outputs, labels).item()

            val_acc = 100 * val_correct / len(self.val_loader.dataset)
            avg_val_loss = val_loss / len(self.val_loader)

            self.history["train_loss"].append(avg_train_loss)
            self.history["train_acc"].append(train_acc)
            self.history["val_loss"].append(avg_val_loss)
            self.history["val_acc"].append(val_acc)

            print(
                f"Epoch {epoch+1}/{EPOCHS} - Train: {train_acc:.2f}% - Val: {val_acc:.2f}%"
            )
        return self.history

    def test_model(self, test_loader=None):
        self.test_loader = test_loader
        # ============================================
        # TEST
        # ============================================
        self.model.eval()
        test_correct = 0
        test_loss = 0

        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.forward(images)
                test_correct += (outputs.argmax(1) == labels).sum().item()
                test_loss += self.criterion(outputs, labels).item()

        test_acc = 100 * test_correct / len(self.test_loader.dataset)
        print(f"\nTest Accuracy: {test_acc:.2f}%")
        test_loss = test_loss / len(self.test_loader)
        return test_acc, test_loss

print("loaded yolo")

loaded yolo


**train loops**

In [28]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torch
import time

model_names = ["yolo", "vgg"]
splits = ["split_42", "split_123"]

for m_name in model_names:
    for split in splits:

        # loading transform fucntions
        if m_name == "vgg":
            train_transform, test_transform = preprocess_for_vgg19()
            classifier = VGG19Classifier()
            print("training vgg19")
        else:
            continue
            # train_transform, test_transform = preprocess_for_yolov5()
            # classifier = YOLOv5Classifier()

        train_dataset = ImageFolder(
            f"/kaggle/working/data/splits/{split}/train", transform=train_transform
        )
        val_dataset = ImageFolder(
            f"/kaggle/working/data/splits/{split}/val", transform=test_transform
        )
        test_dataset = ImageFolder(
            f"/kaggle/working/data/splits/{split}/test", transform=test_transform
        )

        # Create dataloaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

        # def test_gpu_speed(classifier, label="Model"):
        #     # Move model to device
        #     classifier.to(DEVICE)
        #     classifier.eval()
            
        #     # Create 50 batches of fake data (Noise)
        #     dummy_input = torch.randn(16, 3, 224, 224).to(DEVICE)
            
        #     print(f"--- Testing {label} on GPU ---")
        #     start = time.time()
        #     with torch.no_grad():
        #         for _ in range(50):
        #             _ = classifier(dummy_input)
        #     end = time.time()
            
        #     print(f"Finished 50 batches in {end - start:.2f} seconds.")
        #     print(f"Approx {50/(end-start):.2f} batches per second.\n")
        
        # # Run for both
        # test_gpu_speed(YOLOv5Classifier(), "YOLOv5")
        # test_gpu_speed(VGG19Classifier(), "VGG19")

        # print(f"Model: {m_name.upper()}, Split: {split}")

        history = classifier.train_model(
            train_loader=train_loader, val_loader=val_loader
        )
        test_acc, test_loss = classifier.test_model(test_loader=test_loader)

        plot_cross_entropy_loss(
            history, model_name=m_name, split_name=split, test_loss=test_loss
        )
        plot_accuracy_graph(
            history, model_name=m_name, split_name=split, test_acc=test_acc
        )


TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'